# Lesson 2 - Session 1 - String is all you need

### Learning Objectives:

1. Understand that in Bash, all variables are treated as strings by default.
2. Learn how to declare and initialize variables in Bash.
3. Explore how to perform arithmetic operations in Bash and the need for explicit type conversion.
4. Practice using declare and let for type conversion and arithmetic operations.
5. Understand the implications of string-based operations in scripting and how to handle them.

## 1. Understand Bash's variable and Environment variables

Declare and initialize variables in Bash is very simple. You can declare a variable by assigning a value to it. You can also declare a variable by using the declare command. The declare command is used to create variables and assign attributes to them. You can see the example below.

In [26]:
%%bash

textbook="hello world!"
echo $textbook

hello world!


Just type the variable name and assign the value to it. You can also use the declare command to declare a variable. You can see the example below.

In [27]:
%%bash

declare variable="value"
declare -r readonly="readonly"

echo "$variable"
echo "$readonly"

value
readonly


While it seems similar to the first example, there are two noticable differences between using `a=b` and `declare a=b`. First, a=b is global variable, while declare a=b is a local variable. for example, if you declare a variable in a function, it will be a local variable. You can see the example below.

In [88]:
%%bash

function global_variable_func() {
    globvar="hello" # here
    echo "globvar is set to '$globvar' within function"
}

global_variable_func

if [ -z ${globvar+x} ]; then
    echo "globvar is unset in global score"
else
    echo "globvar is set to '$globvar'"
fi

globvar is set to 'hello' within function
globvar is set to 'hello'


In [87]:
%%bash

function local_variable_func() {
    declare localvar="world" # here
    echo "localvar is set to '$localvar' within function"
}

local_variable_func

if [ -z ${localvar+x} ]; then
 echo "localvar is unset on the global scope"
else
 echo "localvar is set to '$localvar'"
fi

localvar is set to 'world' within function
localvar is unset on the global scope


And also, declare command has some additional attributes that can be used to declare variables. The `-r` option is used to declare a variable as read-only. If you try to change the value of a read-only variable, you will get an error like below.

In [28]:
%%bash

declare -r readonly="readonly"
readonly="new value"

bash: line 3: readonly: readonly variable


CalledProcessError: Command 'b'\ndeclare -r readonly="readonly"\nreadonly="new value"\n'' returned non-zero exit status 1.

You can also use the -a and -A option to declare an array and map variable, respectively. You can see the example below. We will discuss arrays in the L3-s1 part later, so don't worry about it now.

#### Array

In [42]:
%%bash

declare -a array=("value1" "value2" "value3")
echo "arr full: ${array[@]}"
echo "arr length: ${#array[@]}"
echo "First value of array: ${array[0]}"

arr full: value1 value2 value3
arr length: 3
First value of array: value1


#### Map

if you're using Bash version 4.0 or later, you can use the -A option to declare a map variable. You can see the example below. We will discuss map in the L3-s1 part later, so don't worry about it now.

If you're using zsh or older versions of Bash, you cannot use the declare command to declare a map variable, so it will raise an error. You can see the example below.

In [43]:
%%bash

declare -A map_arr \
map_arr["name"]="John" \
map_arr["age"]="30" \
map_arr["city"]="New York"

echo "Full map: ${map_arr[@]}"
echo "Map length: ${#map_arr[@]}"
echo "Name: ${map_arr["name"]}"

bash: line 2: declare: -A: invalid option
declare: usage: declare [-afFirtx] [-p] [name[=value] ...]


Full map: 
Map length: 0
Name: 


You can also use the declare command to list variables already declared. You can see the example below.

In [25]:
%%bash

declare | grep SHELL=

SHELL=/bin/zsh


This shows that the SHELL variable is an environment variable. Environment variables are variables that are available to all processes and are used to store information about the system environment. You can see the example below.

In [45]:
%%bash

export ENVVAR_EXAMPLE="example"
echo $ENVVAR_EXAMPLE

example


In [89]:
%%bash

declare -x ENVVAR_EXAMPLE_2="example2"
echo $ENVVAR_EXAMPLE_2

example2


## 2. All variables are treated as strings by default

By default, all variables in Bash are treated as strings. Specifically, all variables in Bash are **untyped**, and if a variable contains only digits, it can be treated as an integer depending on the context. This can be confusing because it differs from other programming languages. Let's see how it works.

In [27]:
%%bash

a=5
b=3
output=$a+$b
echo "result without expr: $output"

result without expr: 5+3


As you can see, you cannot add or multiply the a and b variables just by declaring them and using `$a+$b`. All elements, including `+`, are treated as strings. However, you can perform integer calculations using the following three methods.

## 3. Calculation 

One way to perform integer operations in Bash is by using double brackets `$(())`. By declaring digit-only variables and using operators and variables in the `$((a + b))` format, you can perform arithmetic operations in Bash.

Another way to perform integer operations in Bash is by using the expr command. By declaring digit-only variables and using operators and variables in the expr `$a + $b` format, you can perform arithmetic operations in Bash.

You can also use the let syntax to perform arithmetic operations in Bash. By declaring digit-only variables and using operators and variables in the `let c=$a+$b` format, you can perform arithmetic operations in Bash.

Operators that can be used in arithmetic operations are `+`, `-`, `*`, `/`, `%`, `**`, and `+=`, `-=`, `*=`, `/=`, `%=`, `**=` for assignment operations.


In [52]:
%%bash

a=10
b=5
double_brackets=$(($a + $b))
expr_outputs=$(expr $a \* $b)
divide=$((a / b))
square=$((a ** b))
let plus_equal=$a*$b

echo "result in (()): $double_brackets"
echo "result in expr $expr_outputs"
echo "result in divide: $divide"
echo "result of square: $square"
echo "result of plus equal: $plus_equal"

result in (()): 15
result in expr 50
result in divide: 2
result of square: 100000
result of another square: 
result of plus equal: 50


One thing you should know is that only integers can be used in arithmetic operations. If you try to use floating-point numbers, you will get an error like below. You should use `bc` command to calculate floating-point numbers.

In [31]:
%%bash

a=5.3
b=3.2

double_brackets=$((a + b))
expr_outputs=$(expr $a + $b)

echo "result in (()): $double_brackets"
echo "result in expr $expr_outputs"

bash: line 5: 5.3: syntax error: invalid arithmetic operator (error token is ".3")
expr: not a decimal number: '5.3'


result in (()): 
result in expr 


As you can see, floating points are treated as string, therefore error has raised in the script. but if you use bc, you can calculate floating point numbers.

In [56]:
%%bash

num1=10.3
num2=3.2

product=$(echo "$num1 * $num2" | bc)
quotient=$(echo "scale=2; $num1 / $num2" | bc)
square=$(echo "$num1 ^ 2" | bc)

echo "Product: $product"
echo "Quotient: $quotient"
echo "Square: $square"

Product: 32.9
Quotient: 3.21
Square: 106.0


Unlike other programming languages, Bash does not support floating-point arithmetic. To perform such calculations, you need to use the bc command, a command-line calculator that handles floating-point arithmetic.

While multiplication can be performed using the `*` operator, division requires the scale option in bc. The scale option specifies the number of decimal places in the result. For example, setting `scale=2` ensures that the quotient is rounded to two decimal places. Without this option, division results are truncated to the nearest integer.

In [12]:
%%bash

num1=11
num2=3

quotient=$(echo "$num1 / $num2" | bc)
echo $quotient

3


## 4. Compare values

As I've described above, all values are treated as strings in Bash. Therefore, when you compare two variables, you should be careful. If you compare two variables, Bash will compare them as strings.

You can use -eq for numeric comparison and = for string comparison. If you use -eq for string comparison, It will get error like below.

In [17]:
%%bash

if [ "hi" -eq "bye" ]; then
    echo "true"
fi

bash: line 2: [: hi: integer expression expected


However, it does not mean that you cannot compare numbers and string that contains only digits. In this situation, you can use -eq for comparison because it treat variables as integers. You can see the example below.

In [15]:
%%bash

int=10
str="10"

if [ "$int" -eq "$str" ]; then
  echo "Integer and string are equal"
else
  echo "Integer and string are not equal"
fi


Integer and string are equal


If you want to compare floating-point numbers, you can use the bc chaining command to convert the numbers to integers and then compare them. You can see the example below. 

By the way, we will discuss of chaining command in the future section, so don't worry about it now. We will not use complex chaining command in this section.

In [21]:
%%bash

int=10
float=10.1

if [ "$(echo "$int < $float" | bc)" -eq 1 ]; then
  echo "$float is bigger than $int"
else
  echo "$int is bigger than $float"
fi

10.1 is bigger than 10


## 5. Assginments

The assginment for this session is to create a script that calculates the area of a circle. The script should take the radius of the circle as an argument and output the area of the circle. The formula for calculating the area of a circle is `πr^2`, where `r` is the radius of the circle. You can use the value of `π` as 3.14, and radius as 10.5. scale of the result should be 2.

you should use the `bc` command to calculate the floating-point number.

In [75]:
%%bash

declare -r radius=10.5
declare -r phi=3.14

formula=$(<your-answer-here>)
echo $formula > ./s1-exercise/result.txt

In [ ]:
with open("./s1-exercise/result.txt", "r") as f:
    output = f.read().strip()

with open("./s1-exercise/answer.txt", "r") as f:
    answer = f.read().strip()

assert float(output) == float(answer)
print("You have successfully completed the assignment.")